### One business must link with unique session


###### in that case check Route business map table and run the following script

In [3]:
business_obj = Business.objects.all()
for business in business_obj:
    for session in Session.objects.all():
        route_business_obj = RouteBusinessMap.objects.filter(business_id=business.id, route__session_id=session.id).count()
        if route_business_obj > 1:
            print('error in {}'.format(business.code))


error in 803


### One route must link with one vechile


###### in that case check Route vehicle map table and run the following scrip

In [1]:
route_obj = Route.objects.all()
for route in route_obj:
    route_vehicle_obj = RouteVehicleMap.objects.filter(route_id=route.id).count()
    if route_vehicle_obj > 1:
        print('error in {}'.format(route.id))
print('Executed')

# need to check whe if any route is not assinted with vehicle

Executed


# Check agent user_type is 2 or not

In [12]:
business_obj = BusinessAgentMap.objects.all()
for business in business_obj:
    if business.agent.user_profile.user_type_id != 2:
        print('business_id - ', business.business.id )
        user_profile_obj = UserProfile.objects.get(id=business.agent.user_profile.id)
        user_profile_obj.user_type_id = 2
        user_profile_obj.save()
        print('updated')

### Removing business from temp route

In [1]:
for route in RouteBusinessMap.objects.filter(route__is_temp_route=True):
    print(route.route)
    main_route = RouteTempRouteMap.objects.get(temp_route_id=route.route.id).main_route
    print(main_route)
    route.route_id = main_route.id
    route.save()

###  Check wheather salegroup has sale in customer order

In [7]:
no_count = 0
customer_list = []
for sale_group in ICustomerSaleGroup.objects.filter(date__month=5,icustomer__customer_type_id=1):
    if ICustomerSale.objects.filter(icustomer_sale_group_id=sale_group.id).exists():
        sale_obj = ICustomerSale.objects.filter(icustomer_sale_group_id=sale_group.id, product_id=1)
        print(sale_obj.count())
    else:
#         print('NOe')
        no_count += 1
        customer_list.append(sale_group.icustomer.customer_code)
no_count
# customer_list

0

In [1]:
for business in Business.objects.filter():
    if SaleGroup.objects.filter(date='2020-05-15', business=business).count() > 2:
        print(business.code)

In [ ]:
# Checking double entry


In [ ]:
# Checking double entry
#
import pytz
indian=pytz.timezone('Asia/Kolkata')
for business in Business.objects.filter():
    if SaleGroup.objects.filter(date='2020-05-15', business=business).count() > 2:
        print(business.code)
        for sg in  SaleGroup.objects.filter(date='2020-05-15', business__code=business.code):
            print("Sale Group: id:{} session:{} \ttotal_cost:{}\tdate:{}\tOrdBy:{}".format(sg.id, sg.session.name, sg.total_cost,sg.time_created.astimezone(indian).strftime("%Y-%m-%d %I:%M %P"),sg.ordered_by.first_name ))
            for sale in sg.sale_set.all():
                print("\tsale id:{} cost:{}\t({}:{})".format(sale.id, sale.cost, sale.product.code, sale.count ))



In [ ]:
# Validate Routely report

In [7]:
for r in DailySessionllyRoutellySale.objects.filter(delivery_date="2020-05-10", sold_to="Agent"):
    print("{}\t{}\t{}\t{}\t{}\t{}".format(r.route.name, r.session, r.delivery_date, r.tm500_pkt, r.std500_pkt, r.std250_pkt))

N - IRUGUR_MOR	Mor	2020-05-10	154	5812	8856
4 - SARAVA_EVE	Eve	2020-05-10	0	856	347
D - MTP RO_EVE	Eve	2020-05-10	0	1667	484
R - POOSAR_EVE	Eve	2020-05-10	0	1675	626
I - MARUDH_MOR	Mor	2020-05-10	237	6682	2812
1 - SULUR_MOR	Mor	2020-05-10	96	5307	7411
J - RAMANA_EVE	Eve	2020-05-10	0	1472	1889
H - MADHUK_EVE	Eve	2020-05-10	0	483	212
R - POOSAR_MOR	Mor	2020-05-10	320	4959	2682
P - THONDA_MOR	Mor	2020-05-10	54	3220	1424
A - PODANU_EVE	Eve	2020-05-10	0	156	144
M - POLLAC_MOR	Mor	2020-05-10	0	1354	2928
B - SAIBAB_EVE	Eve	2020-05-10	0	1002	264
O2 - TIRUP_MOR	Mor	2020-05-10	0	3332	5796
L - GANAPA_EVE	Eve	2020-05-10	0	486	371
L - GANAPA_MOR	Mor	2020-05-10	110	6875	4837
5 - SOMANU_MOR	Mor	2020-05-10	143	6089	6802
5 - SOMANU_EVE	Eve	2020-05-10	0	706	1068
E - TOWN_EVE	Eve	2020-05-10	230	102	0
G - SELVAP_MOR	Mor	2020-05-10	85	5223	3162
W - ROUTE_MOR	Mor	2020-05-10	138	5755	6477
B - SAIBAB_MOR	Mor	2020-05-10	132	6604	4880
O - TIRUPP_MOR	Mor	2020-05-10	0	2206	4808
S - GANDHI_EVE	Eve	2020-05-10	0	160

In [37]:
# List Agents without phone/pan numbers
#

#collect active agents
active_business_ids = SaleGroup.objects.all().values_list('business_id', flat=True)
print(active_business_ids)
active_agent_ids = BusinessAgentMap.objects.filter(business_id__in=active_business_ids).values_list('agent_id', flat=True)



print(active_agents)

sl = 0
for ag in Agent.objects.filter(id__in=active_agent_ids):
    booth = BusinessAgentMap.objects.filter(agent=ag)[0].business
    sl = sl+1
    bank = ag.agentbankdetail
    
    print("{},{},{},{},{},{},{},{},{},{},{},#,{},{},{},{},{}".format(sl, booth.code, booth.zone.name, booth.business_type.name, ag.id, ag.agent_code, ag.user_profile.user.first_name, ag.user_profile.user.last_name, ag.user_profile.mobile, ag.aadhar_number, ag.pan_number, bank.bank, bank.branch, bank.ifsc_code,bank.account_holder_name, bank.account_number ))
    

<QuerySet [1786, 1786, 1740, 1611, 1611, 1816, 2577, 1712, 1929, 1929, 1977, 1867, 1736, 1736, 1958, 1626, 1626, 1610, 1610, 1665, '...(remaining elements truncated)...']>
<QuerySet [1288, 1289, 1291, 1292, 1293, 1294, 1295, 1297, 1298, 1299, 1300, 1302, 1303, 1304, 1305, 1306, 1287, 1309, 1310, 1311, '...(remaining elements truncated)...']>
1,415,EAST,Booth,1328,PR235504,PREMKUMAR,M,9600967794,None,None,#,Axis Bank,Coimbatore,UTIB0000090,PREMKUMAR_M,916010037571699
2,458,EAST,Booth,1336,AN993868,ANANDAKUMAR,,0,None,None,#,Axis Bank,Coimbatore,UTIB0000090,ANANDAKUMAR_,911010013118820
3,481,EAST,Booth,1341,DH687000,DHANALAKSHIMI,,0,None,None,#,Axis Bank,Coimbatore,UTIB0000090,DHANALAKSHIMI_,911010012901120
4,639,EAST,Booth,1349,SA850957,SATHIYASEELAN ,S,0,None,None,#,Axis Bank,Coimbatore,UTIB0000090,SATHIYASEELAN _S,911010019076676
5,13,EAST,Booth,1287,1721,DHANALAKSHIMI,,0,None,AJTPD2558A,#,Axis Bank,Coimbatore,UTIB0000090,DHANALAKSHIMI_,911010012901120
6,14,EAST,Booth,1288,719,RAGHAVA

91,2208,EAST,Booth,1468,3027,M,Jerina,9865778980,None,AYMPJ4377M,#,Axis Bank,Coimbatore,UTIB0000090,M_Jerina,917010025874626
92,2211,EAST,Booth,1470,3030,K,Nithya,9842273055,None,BFFPN9486Q,#,Axis Bank,Coimbatore,UTIB0000090,K_Nithya,917010042154273
93,2229,EAST,Booth,1473,3051,K,Indirani,9003567561,None,AETPI0220B,#,Axis Bank,Coimbatore,UTIB0000090,K_Indirani,917010023660724
94,3035,EAST,Govt Institute,1516,7677,Air force Admn - College,,0,None,None,#,bank name,branch name,ifsc code,Air force Admn - College_,1234
95,2245,EAST,Booth,1476,33064,MANOHAR,,0,None,AYEPM5556F,#,Axis Bank,Coimbatore,UTIB0000090,MANOHAR_,917010071081290
96,2220,EAST,Parlour,1472,33040,YesodhaSri,,0,None,AFNPY6945E,#,Axis Bank,Coimbatore,UTIB0000090,YesodhaSri_,918010026321139
97,2274,EAST,Booth,1480,33093,D,CHANDRA,9942744174,None,COCPD5767D,#,Axis Bank,Coimbatore,UTIB0000090,D_CHANDRA,918010013247833
98,2285,EAST,Booth,1482,33105,V SUGANYA,,0,None,IBUPS2894P,#,Axis Bank,Coimbatore,UTIB0000090,V SUGANYA_,91701

185,1511,WEST,Booth,1657,1682,JAYASENTHILAN,V,0,None,AGAPJ8390A,#,Axis Bank,Coimbatore,UTIB0000090,JAYASENTHILAN_V,911010013120111
186,1538,WEST,Booth,1661,1712,JOSEPH JAYAKUMAR,J,0,None,AGZPJ9893M,#,Axis Bank,Coimbatore,UTIB0000090,JOSEPH JAYAKUMAR_J,911010013113812
187,1540,WEST,Booth,1662,121,JANAKARAJ,A,0,None,AGIPA9408N,#,Axis Bank,Coimbatore,UTIB0000090,JANAKARAJ_A,911010037834287
188,1559,WEST,Booth,1664,1735,AMBUJAM,R,0,None,AGNPR0991A,#,Axis Bank,Coimbatore,UTIB0000090,AMBUJAM_R,911010012903401
189,1597,WEST,Booth,1665,1792,RAMKUMAR,,0,None,AHKPR5616G,#,Axis Bank,Coimbatore,UTIB0000090,RAMKUMAR_,911010012981993
190,1614,WEST,Booth,1668,1824,NANDHINIDEVI,G,0,None,AEAPN6794G,#,Axis Bank,Coimbatore,UTIB0000090,NANDHINIDEVI_G,911010014805093
191,1625,WEST,Booth,1670,1835,NAGARAJ,K,0,None,AGDPN9415B,#,Axis Bank,Coimbatore,UTIB0000090,NAGARAJ_K,911010019513117
192,1708,WEST,Booth,1676,1959,RAMAKRISHNAN,S,0,None,AMZPR4661B,#,Axis Bank,Coimbatore,UTIB0000090,RAMAKRISHNAN_S,91101001313

278,365,SOUTH,Booth,1838,727,ABOOTHAHEER,,0,None,AGAPA3344F,#,Axis Bank,Coimbatore,UTIB0000090,ABOOTHAHEER_,911010015168081
279,368,SOUTH,Booth,1839,848,ANNADURAI,S,0,None,AHBPA1286G,#,Axis Bank,Coimbatore,UTIB0000090,ANNADURAI_S,911010016087235
280,402,SOUTH,Booth,1841,2521,MENAKA,S,0,None,AVCPM9699N,#,Axis Bank,Coimbatore,UTIB0000090,MENAKA_S,911010012896772
281,422,SOUTH,Booth,1843,730,SURESH KUMAR,H,0,None,BCUPS5299E,#,Axis Bank,Coimbatore,UTIB0000090,SURESH KUMAR_H,911010035627481
282,432,SOUTH,Booth,1844,2615,MANIVANNAN,V,0,None,BIQPM0497L,#,Axis Bank,Coimbatore,UTIB0000090,MANIVANNAN_V,911010019385666
283,622,SOUTH,Booth,1846,981,KANAGA JESURAJ,,0,None,AQTPK0258P,#,Axis Bank,Coimbatore,UTIB0000090,KANAGA JESURAJ_,911010012898901
284,623,SOUTH,Booth,1847,990,INDIRANI,K,0,None,AAOPI5743A,#,Axis Bank,Coimbatore,UTIB0000090,INDIRANI_K,911010047104682
285,8101,WEST,Own parlour,1774,8101,UNION PARLOUR,,0,None,None,#,bank name,branch name,ifsc code,UNION PARLOUR_,1234
286,363,SOUTH,Boo

371,453,POLLACHI,Booth,2065,459,RAMACHANDRAN,L,0,None,AHHPR4428B,#,Axis Bank,Coimbatore,UTIB0000090,RAMACHANDRAN_L,911010012753536
372,490,POLLACHI,Booth,2066,597,RAMESH,S,0,None,AHFPR6429L,#,Axis Bank,Coimbatore,UTIB0000090,RAMESH_S,911010012760628
373,715,POLLACHI,Booth,2072,1176,KOILRAJ,P,0,None,ANIPK5891L,#,Axis Bank,Coimbatore,UTIB0000090,KOILRAJ_P,911010012757965
374,1205,POLLACHI,Booth,2075,2105,UTHAYAKUMAR,R,0,None,AAUPU6461D,#,Axis Bank,Coimbatore,UTIB0000090,UTHAYAKUMAR_R,911010012756384
375,1223,POLLACHI,Booth,2079,2123,KOUSALYA,V,0,None,AROPK8908L,#,Axis Bank,Coimbatore,UTIB0000090,KOUSALYA_V,911010012754021
376,1225,POLLACHI,Booth,2081,2125,LAKSHMANAN,,0,None,ADTPL0033C,#,Axis Bank,Coimbatore,UTIB0000090,LAKSHMANAN_,911010012756302
377,3145,SOUTH,Govt Institute,2020,7509,RAPID ACTION FORCE-CRPF VELLAL,,0,None,None,#,bank name,branch name,ifsc code,RAPID ACTION FORCE-CRPF VELL_,1234
378,5526,SOUTH,Booth,2056,1857,ASHOK KUMAR,T,0,None,ATFPK7923C,#,bank name,branch name,ifsc 

464,1509,NORTH,Booth,2250,1679,THIRUMAL NADAR,R,0,None,AHPPT3839A,#,Axis Bank,Coimbatore,UTIB0000090,THIRUMAL NADAR_R,911010012756975
465,1520,NORTH,Booth,2251,1694,BALASUBRAMANIAN,P,0,None,AIIPB5071E,#,Axis Bank,Coimbatore,UTIB0000090,BALASUBRAMANIAN_P,911010012623257
466,1554,NORTH,Booth,2253,1730,MANI,K,0,None,ALIPM5591M,#,Axis Bank,Coimbatore,UTIB0000090,MANI_K,911010012612930
467,1580,NORTH,Booth,2256,2741,Mariya Jagan,J,0,None,AOEPM8469H,#,Axis Bank,Coimbatore,UTIB0000090,Mariya Jagan_J,913010022564760
468,1583,NORTH,Booth,2257,1765,LAKSHMI,G,0,None,ADKPL3776H,#,Axis Bank,Coimbatore,UTIB0000090,LAKSHMI_G,911010012611940
469,1601,NORTH,Booth,2260,2541,SURESH LINGAM ,M,0,None,CFZPS7567E,#,Axis Bank,Coimbatore,UTIB0000090,SURESH LINGAM _M,911010021847499
470,1617,NORTH,Booth,2261,1827,RAJAMANICKEM,M,0,None,AJGPR4570F,#,Axis Bank,Coimbatore,UTIB0000090,RAJAMANICKEM_M,911010012656309
471,1638,NORTH,Booth,2263,1853,IMRANKHAN,,0,None,AAZPI5554Q,#,Axis Bank,Coimbatore,UTIB0000090,IMRANKH

557,1140,MTP,Booth,2473,2040,KOUSALYA,S,9047050706,None,AZTPK2353C,#,Axis Bank,Coimbatore,UTIB0000090,KOUSALYA_S,911010012757697
558,1148,MTP,Booth,2476,2048,PREMILA,N,0,None,BJHPP9601F,#,Axis Bank,Coimbatore,UTIB0000090,PREMILA_N,911010012755901
559,8109,NORTH,Own parlour,2434,8109,VOC PARK PARLOUR,,0,None,None,#,bank name,branch name,ifsc code,V_,1234
560,4203,NORTH,Govt Institute,2419,9644,ASST-COMR COPRN (MTP BUS STAND,,0,None,None,#,bank name,branch name,ifsc code,ASST-COMR_,1234
561,4205,NORTH,Govt Institute,2421,7919,ASST-COMR COPRN (MANIAKARAPALA,,0,None,None,#,bank name,branch name,ifsc code,ASST-COMR_,1234
562,1107,MTP,Booth,2459,2059,SURESH KUMAR ,D,0,None,DCGPS5644F,#,Axis Bank,Coimbatore,UTIB0000090,SURESH KUMAR _D,913010032003080
563,4223,NORTH,Private Institute,2422,4223,Iswariya Fertility Centre,,0,None,None,#,bank name,branch name,ifsc code,Iswariya Fertility Centre_,1234
564,4248,NORTH,Private Institute,2427,4248,SERENE,,0,None,None,#,bank name,branch name,ifsc code,S

649,2644,NORTH,Booth,2637,2637,Azhagudurai,Azhagudurai,1234514234,124235234213,None,#,Not Given,Not Given,Not Given,Azhagudurai,131232131
650,2646,NORTH,Booth,2638,33356,Joseph jeromay,G,9121314151,111111111111,None,#,Not Given,Not Given,Not Given,Joseph jeromay,1111
651,6019,CHENNAI Aavin,Other unions,2639,20230,TCMPF CHENNAI,L,1231231231,111111111111,None,#,Not Given,Not Given,Not Given,Not Given,11
652,9024,SOUTH,Whole Sale Dealer,2640,T 506391_WS,T SURESH KUMAR,,0,None,None,#,bank name,branch name,ifsc code,T SURESH KUMAR_,1234
653,2656,EAST,Booth,2642,33366,Sasireka,velumani,1111111118,210510013320,GBNPS1598R,#,Kotak Mahindra Bank,Vellalore,KMB,Sasireka,5812141194
654,2658,EAST,Booth,2641,33368,Jenifer,vincent,1111111119,650040996915,BATPJ4678M,#,bank of india,ramanathapuram,BKID0008150,Jenifer V,815010110009833
655,9025,SOUTH,Whole Sale Dealer,2643,M646562_WS,Velumani, M,0,None,None,#,Axis Bank,Coimbatore,UTIB0000090,M_Velumani,915010044624578
656,1,WEST,Own parlour,1576,612,UNIO

743,2611,NORTH,Booth,2576,33319,Shanthi,P,0,None,BVOPS3872E,#,CORPORATION-GKNM,GKNM,CORP0000650,Shan_,510101006980597
744,2619,EAST,Booth,2569,33328,G,,1010101111,101020102012,BCAPK6639J,#,IOB-VADAVALLI,VADAVALLI,IOBA0001745,Nan,174501000010996
745,2645,WEST,Booth,2644,33355,Saravanan,G,8883335469,111111111111,BOGBS2590C,#,not given,not given,12232113132,G Saaravanan,11111111


### Check id booth is. not map with route


In [1]:
mor_session = []
eve_session = []
for business in Business.objects.filter():
    if not RouteBusinessMap.objects.filter(route__session_id=1, business=business).exists():
        mor_session.append(business.code)
    if not RouteBusinessMap.objects.filter(route__session_id=2, business=business).exists():
        eve_session.append(business.code)

In [2]:
from io import BytesIO

import requests
import pandas as pd

google_sheet = 'https://docs.google.com/spreadsheets/d/14HVoAqPEANJA_Q-tQudnCSUsCgjgMreMFGZCg7l5OwE/edit?usp=sharing'
r = requests.get(google_sheet)
data = r.content
data

b'<!DOCTYPE html><html lang="en-GB"><head><script nonce="hVSEpXL/ofSy2UjKHLFJoA">var DOCS_timing={}; DOCS_timing[\'pls\']=new Date().getTime();</script><meta property="og:title" content="test_data"><meta property="og:type" content="article"><meta property="og:site_name" content="Google Docs"><meta property="og:url" content="https://docs.google.com/spreadsheets/d/14HVoAqPEANJA_Q-tQudnCSUsCgjgMreMFGZCg7l5OwE/edit?usp=sharing&amp;usp=embed_facebook"><meta property="og:image" content="https://lh3.googleusercontent.com/tenRHSgl46LkqiMMS3aG9W3Pabxk7Il8oVQ0ui9HLkw5NOa7DuscK_Ko1qwVZhab3GSyFO5RCszA3A=w1200-h630-p"><meta property="og:image:width" content="1200"><meta property="og:image:height" content="630"><meta property="og:description" content="Sheet1\n\nslno,class,tn,ap,ka,kl,tl\n1,1,1234,26465,2634,2443,2634\n2,2,3454,1234,2634,2634,1234\n3,3,26465,3446,1234,2634,2634\n4,4,26465,1234,2634,26465,234324\n5,5,654654,26465,26465,5454,3232\n6,6,1234,26465,567,2634,26465\n7,7,6876,1234,26465,5454

In [5]:
import pandas as pd
import numpy as np
import requests
import io

try:
    from StringIO import StringIO
except ImportError:
    
    from io import StringIO
    from io import BytesIO
from decimal import Decimal
google_sheet_url = 'https://docs.google.com/spreadsheets/d/14HVoAqPEANJA_Q-tQudnCSUsCgjgMreMFGZCg7l5OwE/export?format=xlsx'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_excel(BytesIO(data),sheet_name='Sheet1')
df

<ipython-input-5-17948176c4bf>:16: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel(BytesIO(data),sheet_name='Sheet1')


,slno,class,tn,ap,ka,kl,tl
0,1,1,1234,26465,2634,2443,2634
1,2,2,3454,1234,2634,2634,1234
2,3,3,26465,3446,1234,2634,2634
3,4,4,26465,1234,2634,26465,234324
4,5,5,654654,26465,26465,5454,3232
5,6,6,1234,26465,567,2634,26465
6,7,7,6876,1234,26465,54545,32323
7,8,8,656,886,1234,7676,1234
8,9,9,345435,5785,876,4554,26465
9,10,10,2634,8666,393,2634,3232
